<a href="https://colab.research.google.com/github/joaomrossetto/DYN_MST_BRSE/blob/master/CODE_Redes_de_correla%C3%A7%C3%A3o_antiBolsominions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **`Subindo e Configurando o Ambiente Hadoop e Spark 2.4.1`**


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -P /content/ https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!!pip install -q findspark

--2019-10-21 22:57:02--  https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 163.172.17.199
Connecting to archive.apache.org (archive.apache.org)|163.172.17.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230778742 (220M) [application/x-gzip]
Saving to: ‘/content/spark-2.4.1-bin-hadoop2.7.tgz’

spark-2.4.1-bin-had 100%[===================>] 220.09M  28.1MB/s    in 7.9s    

2019-10-21 22:57:10 (27.8 MB/s) - ‘/content/spark-2.4.1-bin-hadoop2.7.tgz’ saved [230778742/230778742]



[]

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init('/content/spark-2.4.1-bin-hadoop2.7')
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("Redes_de_Correlacao_Ativos") \
    .config('spark.yarn.executor.memoryOverhead', '4096') \
    .getOrCreate()

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

### Importando libs

In [0]:
# !pip install koalas
# import databricks.koalas as ks
from pyspark.sql import functions as F
from pyspark.sql.window import Window

## Realizando mount com dados de forma governada

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **Importando dados de cotação historica**

*   List item
*   List item



In [0]:
path ="drive/Shared\ drives/RP\ 2/Dados\ Tratados/COTAHIST_A2019.TXT"
df_txt = spark.read.text(path,wholetext=False,lineSep='\n')

In [0]:
df=df_txt.select(
    df_txt.value.substr(1,2).alias('TIPO_REG'),
    df_txt.value.substr(3,8).alias('DATA_PREG'),
    df_txt.value.substr(11,2).alias('COD_BDI'),
    df_txt.value.substr(13,12).alias('COD_NEG'),
    df_txt.value.substr(25,3).alias('TPMERC'),
    df_txt.value.substr(28,12).alias('NOMRES'),
    df_txt.value.substr(40,10).alias('ESPECI'),
    df_txt.value.substr(50,3).alias('PRAZUT'),
    df_txt.value.substr(53,4).alias('MODREF'),
    df_txt.value.substr(57,11).alias('PREC_ABER'),
    df_txt.value.substr(70,11).alias('PREC_MAX'),
    df_txt.value.substr(83,11).alias('PREC_MIN'),
    df_txt.value.substr(96,11).alias('PREC_MED'),
    df_txt.value.substr(109,11).alias('PREC_ULT'),
    df_txt.value.substr(122,11).alias('PREC_COMPR'),
    df_txt.value.substr(135,11).alias('PREC_VEND'),
    df_txt.value.substr(148,5).alias('TOTAL_NEG'),
    df_txt.value.substr(153,18).alias('QT_TOTAL'),
    df_txt.value.substr(171,16).alias('VOL_TOTAL'),
    df_txt.value.substr(189,11).alias('PREC_EXERC'),
    df_txt.value.substr(202,1).alias('IND_OPC'),
    df_txt.value.substr(203,8).alias('DT_VENC'),
    df_txt.value.substr(211,7).alias('FAT_COT'),
    df_txt.value.substr(218,7).alias('PTO_EXE'),
    df_txt.value.substr(231,12).alias('COD_ISIN'),
    df_txt.value.substr(243,3).alias('DIS_MES')
)

# Tratamento dos dados

In [0]:
def regUnp(df_wk,name):
  df_wk.registerTempTable(name)
  df_wk.unpersist()
  return df_wk.printSchema()

In [0]:
from pyspark.sql.functions import to_timestamp
df=df.withColumn("DATA_PREG", to_timestamp('DATA_PREG','yyyyMMdd'))

In [0]:
# df=df.na.drop()

In [0]:
regUnp(df,"WK01")

root
 |-- TIPO_REG: string (nullable = true)
 |-- DATA_PREG: timestamp (nullable = true)
 |-- COD_BDI: string (nullable = true)
 |-- COD_NEG: string (nullable = true)
 |-- TPMERC: string (nullable = true)
 |-- NOMRES: string (nullable = true)
 |-- ESPECI: string (nullable = true)
 |-- PRAZUT: string (nullable = true)
 |-- MODREF: string (nullable = true)
 |-- PREC_ABER: string (nullable = true)
 |-- PREC_MAX: string (nullable = true)
 |-- PREC_MIN: string (nullable = true)
 |-- PREC_MED: string (nullable = true)
 |-- PREC_ULT: string (nullable = true)
 |-- PREC_COMPR: string (nullable = true)
 |-- PREC_VEND: string (nullable = true)
 |-- TOTAL_NEG: string (nullable = true)
 |-- QT_TOTAL: string (nullable = true)
 |-- VOL_TOTAL: string (nullable = true)
 |-- PREC_EXERC: string (nullable = true)
 |-- IND_OPC: string (nullable = true)
 |-- DT_VENC: string (nullable = true)
 |-- FAT_COT: string (nullable = true)
 |-- PTO_EXE: string (nullable = true)
 |-- COD_ISIN: string (nullable = true)

## Tratando tipos dos dados

In [0]:
df=spark.sql(
"""
SELECT
   rtrim(ltrim(COD_NEG)) as COD_NEG
  ,rtrim(ltrim(NOMRES)) as NOMRES
  ,DATA_PREG as DATA_PREG
  ,cast(COD_BDI as int) as COD_BDI
  ,cast(TPMERC as int) as TPMERC
  ,replace(cast(ESPECI as string)," ","") as ESPECI
  ,cast(PREC_ABER as bigint) as PREC_ABER
  ,cast(PREC_MAX as bigint) as PREC_MAX
  ,cast(PREC_MIN as bigint) as PREC_MIN
  ,cast(PREC_MED as bigint) as PREC_MED
  ,cast(PREC_ULT as bigint) as PREC_ULT
  ,cast(PREC_COMPR as bigint) as PREC_COMPR
  ,cast(PREC_VEND as bigint) as PREC_VEND
  ,cast(TOTAL_NEG as int) as TOTAL_NEG
  ,cast(QT_TOTAL as bigint) as QT_TOTAL
  ,cast(VOL_TOTAL as bigint) as VOL_TOTAL
  ,cast(PREC_EXERC as bigint) as PREC_EXERC
  ,cast(PTO_EXE as bigint) as PTO_EXE
FROM
  WK01
""")

# Filtra ativos de lote Padrão
df=df.filter("COD_BDI=2")

regUnp(df,"WK02")

root
 |-- COD_NEG: string (nullable = true)
 |-- NOMRES: string (nullable = true)
 |-- DATA_PREG: timestamp (nullable = true)
 |-- COD_BDI: integer (nullable = true)
 |-- TPMERC: integer (nullable = true)
 |-- ESPECI: string (nullable = true)
 |-- PREC_ABER: long (nullable = true)
 |-- PREC_MAX: long (nullable = true)
 |-- PREC_MIN: long (nullable = true)
 |-- PREC_MED: long (nullable = true)
 |-- PREC_ULT: long (nullable = true)
 |-- PREC_COMPR: long (nullable = true)
 |-- PREC_VEND: long (nullable = true)
 |-- TOTAL_NEG: integer (nullable = true)
 |-- QT_TOTAL: long (nullable = true)
 |-- VOL_TOTAL: long (nullable = true)
 |-- PREC_EXERC: long (nullable = true)
 |-- PTO_EXE: long (nullable = true)



## Cria Janela para analise

In [0]:
DFW = Window.partitionBy('COD_NEG','DATA_PREG')

## Range de datas disponiveis na base

In [0]:
min_max_n = df.agg(F.min(df.DATA_PREG),F.max(df.DATA_PREG),F.countDistinct(df.DATA_PREG)).collect()[0]
min_date = min_max_n[0]
max_date = min_max_n[1]
ndias_fun = min_max_n[2]
print(min_date,max_date,ndias_fun)

2019-01-02 00:00:00 2019-09-06 00:00:00 171


## Filtros de Elegibilidade de Ativo

*

*   Foi eleito o ativo mais líquido da empresa
    *    http://www.nefin.com.br/illiquidity_index.html
*   Tenha sido negociado em pelo menos 80% dos dias de funcionamento da bolsa
    *   No periodo selecionado
*   Foram considerados aos valores de fechamento diários. Preferiu-se ações do tipo PN –
Preferenciais Nominais a ON – Ordinárias Nominais, como critério de desempate


#### Eligibility criteria (NEFIN Methodology)
##### A stock traded in BOVESPA is considered “eligible” for year ***t*** ifit meets 3 criteria:
  * •The stock is the most traded stock of the firm (the one with the highest traded volume during last year);
  * •The stock was traded in more than 80% of the days in year t-1with volume greater than R$ 500.000,00 per day. In case the stock was listed in year t-1, the period considered goes from the listing day to the last day of the year;
  * •The stock was initially listed prior to December of year t-1



# Negociado 80% dos dias de funcionamento da bolsa

In [0]:
df=df.withColumn('count_neg', F.count('COD_NEG').over(DFW))

In [0]:
DFW = Window.partitionBy('COD_NEG','NOMRES')

In [0]:
df=df.withColumn('sum_count_neg', F.sum('count_neg').over(DFW))

In [0]:
df=df.withColumn("percent_func",(F.col("sum_count_neg")*100)/F.lit(ndias_fun)).filter("percent_func >= 80.0")

root
 |-- COD_NEG: string (nullable = true)
 |-- NOMRES: string (nullable = true)
 |-- DATA_PREG: timestamp (nullable = true)
 |-- COD_BDI: integer (nullable = true)
 |-- TPMERC: integer (nullable = true)
 |-- ESPECI: string (nullable = true)
 |-- PREC_ABER: long (nullable = true)
 |-- PREC_MAX: long (nullable = true)
 |-- PREC_MIN: long (nullable = true)
 |-- PREC_MED: long (nullable = true)
 |-- PREC_ULT: long (nullable = true)
 |-- PREC_COMPR: long (nullable = true)
 |-- PREC_VEND: long (nullable = true)
 |-- TOTAL_NEG: integer (nullable = true)
 |-- QT_TOTAL: long (nullable = true)
 |-- VOL_TOTAL: long (nullable = true)
 |-- PREC_EXERC: long (nullable = true)
 |-- PTO_EXE: long (nullable = true)
 |-- count_neg: long (nullable = false)
 |-- sum_count_neg: long (nullable = true)
 |-- percent_func: double (nullable = true)



# ATIVOS QUE AINDA NÃO FORAM ELEITOS

In [0]:
DFW = Window.partitionBy('NOMRES')
df=df.withColumn('desemp', F.count('*').over(DFW))
regUnp(df,"ativos_preleitos")

root
 |-- COD_NEG: string (nullable = true)
 |-- NOMRES: string (nullable = true)
 |-- DATA_PREG: timestamp (nullable = true)
 |-- COD_BDI: integer (nullable = true)
 |-- TPMERC: integer (nullable = true)
 |-- ESPECI: string (nullable = true)
 |-- PREC_ABER: long (nullable = true)
 |-- PREC_MAX: long (nullable = true)
 |-- PREC_MIN: long (nullable = true)
 |-- PREC_MED: long (nullable = true)
 |-- PREC_ULT: long (nullable = true)
 |-- PREC_COMPR: long (nullable = true)
 |-- PREC_VEND: long (nullable = true)
 |-- TOTAL_NEG: integer (nullable = true)
 |-- QT_TOTAL: long (nullable = true)
 |-- VOL_TOTAL: long (nullable = true)
 |-- PREC_EXERC: long (nullable = true)
 |-- PTO_EXE: long (nullable = true)
 |-- count_neg: long (nullable = false)
 |-- sum_count_neg: long (nullable = true)
 |-- percent_func: double (nullable = true)
 |-- desemp: long (nullable = false)



N = número total de negócios no mercado a vista B3 (lote-padrão).

In [0]:
N=spark.sql(
"""
SELECT
   sum(QT_TOTAL)
FROM
  ativos_preleitos
""").collect()[0][0]

100002280800

# Eleger ativo mais liquido (Com maior indice de Negociabilidade)
O índice de negociabilidade é calculado pela seguinte fórmula:

$IN=\frac{\sum_{i=1}^p\sqrt[3]{\frac{N_a}{N}}*(\frac{V_a}{V})}{P}$



Onde:

IN = índice de negociabilidade.

$N_a$ = número de negócios com o ativo $a$ no mercado a vista (lote padrão)

N = número total de negócios no mercado a vista B3 (lote-padrão).

$V_a$ = volume financeiro gerado pelos negócios com o ativo $a$ no mercado a vista (lote-padrão).

V = volume financeiro total do mercado a BM&FBOVESPA (lote-padrão).

P = número total de pregões no período.

In [0]:
Na=QT_TOTAL ,COD_NEG
N= SUM(QT_TOTAL)

# Calculo de Retorno

$R_{t+1}=\frac{pt+1}{pt}-1 $

Ou em Logaritmo neperiano

$\ln{R_{t+1}}=\ln{p_{t+1}}-\ln{p_{t}} $

### Onde:

$t=período$

$R_{t+1}=RetornoNoDia$

$p_{t+1}=PreçoNoDia$

$p_{t}=PreçoNoDiaAnterior$

In [0]:
def retorno(ativo,dia):

# Correlação de Pearson

$\rho=\frac{\sum_{i=1}^n(xi-\overline{x})(yi-\overline{y})}{\sqrt{\sum_{i=1}^n(xi-\overline{x})^2}*\sqrt{\sum_{i=1}^n(yi-\overline{y})^2}}$

# Distancia da Correlação

$dxy=\sqrt{2(1-\rho xy(\Delta t))}$

In [0]:
spark.sql(
"""
SELECT
  COUNT(DISTINCT COD_NEG)
  ,COUNT(DISTINCT NOMRES)
--  ,NOMRES
--  ,COD_NEG
--  ,NOMRES
FROM
  WK01
"""
).show(100)

+-----------------------+----------------------+
|count(DISTINCT COD_NEG)|count(DISTINCT NOMRES)|
+-----------------------+----------------------+
|                  36956|                  1403|
+-----------------------+----------------------+



In [0]:
spark.sql(
"""
SELECT
  NOMRES
  ,COUNT(COD_NEG) AS COD_NEG
--  ,VOL_TOTAL
FROM
  WK01
GROUP BY
  NOMRES
"""
).show(100,truncate=False)

+------------+-------+
|NOMRES      |COD_NEG|
+------------+-------+
|SCHWAB      |29     |
|VERIZON     |196    |
|MRVE    /ATZ|9      |
|DIRRE    /ED|1      |
|CCRO    /ED |18     |
|SER EDUCA   |418    |
|FII KINEA RI|171    |
|FII MALLS BP|177    |
|MRVE  FM    |510    |
|WEGE  FM/ED |91     |
|SBSPE  FM/EJ|17     |
|PAR AL BAHIA|136    |
|COCA COLA   |64     |
|OI          |1695   |
|MEDTRONIC   |6      |
|RIOSULENSE  |38     |
|FII TOURMALE|1      |
|FII RIOB ED |90     |
|CIEL    /EJ |219    |
|MGLU    /EJ |1      |
|PCAR        |496    |
|FII CAPI SEC|171    |
|MMX MINER   |641    |
|FII HEDGEBS |171    |
|AVON        |117    |
|ELECTR ARTS |1      |
|HYPEE  FM/EJ|90     |
|ARZZE       |9      |
|FII CAMPUSFL|171    |
|FII EXCELLEN|171    |
|IT NOW IDIV |355    |
|BBDCE       |4443   |
|IBOVE  FM   |7601   |
|BRAP    /EDJ|9      |
|AMPLA ENERG |57     |
|BANK AMERICA|227    |
|TAESA       |1201   |
|MILLS       |576    |
|DIRECIONAL  |401    |
|BERKSHIRE   |216    |
|SCHULZ    

In [0]:
df.show()
# dias_ativo_agg=spark.sql(
# """
# SELECT
#    COD_NEG
#    ,DATA_PREG
#    ,COUNT(DISTINCT COD_NEG) as count_neg
#    ,max(NOMRES) as NOMRES
# FROM
#   WK02
# GROUP BY
#   COD_NEG
#   ,DATA_PREG
# """).groupBy('COD_NEG','NOMRES').agg({'count_neg':'sum'})

+------------+------------+-------------------+-------+------+-------+---------+--------+--------+--------+--------+----------+---------+---------+--------+---------+----------+-------+
|     COD_NEG|      NOMRES|          DATA_PREG|COD_BDI|TPMERC| ESPECI|PREC_ABER|PREC_MAX|PREC_MIN|PREC_MED|PREC_ULT|PREC_COMPR|PREC_VEND|TOTAL_NEG|QT_TOTAL|VOL_TOTAL|PREC_EXERC|PTO_EXE|
+------------+------------+-------------------+-------+------+-------+---------+--------+--------+--------+--------+----------+---------+---------+--------+---------+----------+-------+
|AALR3       |ALLIAR      |2019-01-02 00:00:00|      2|    10|   ONNM|       13|      13|      13|      13|      13|        13|       13|      856|  264200|  3523962|         0|      0|
|AAPL34      |APPLE       |2019-01-02 00:00:00|      2|    10|    DRN|       60|      60|      59|      60|      60|        59|       60|       45|   16800|  1015969|         0|      0|
|ABBV34      |ABBVIE      |2019-01-02 00:00:00|      2|    10|    DRN|

In [0]:
from google.colab import files
files.download('drive/Shared drives/RP 2/Materiais/Dissertacao+folhaaprova.pdf')

In [0]:
!ls drive/Shared\ drives/RP\ 2/Materiais/Dissertacao+folhaaprova.pdf

'drive/Shared drives/RP 2/Materiais/Dissertacao+folhaaprova.pdf'


# PyVis - Biblioteca Gráfica
https://pyvis.readthedocs.io/en/latest/tutorial.html
# Documentação
https://pyvis.readthedocs.io/en/latest/documentation.html


In [0]:
! pip install pyvis

In [0]:
! pip install ipython

In [0]:
from pyvis.network import Network
import pandas as pd
import IPython

got_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
got_net.barnes_hut()
got_data = pd.read_csv("https://www.macalester.edu/~abeverid/data/stormofswords.csv")

sources = got_data['Source']
targets = got_data['Target']
weights = got_data['Weight']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst, value=w)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]])

got_net.show("gameofthrones.html")
IPython.display.HTML(filename='gameofthrones.html')